## IMPORTS

In [12]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ERROR: Failed on file: /content/drive/MyDrive/TestFastAPI_v2.py
Traceback (most recent call last):
  File "/usr/local/bin/pipreqs", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/dist-packages/pipreqs/pipreqs.py", line 470, in main
    init(args)
  File "/usr/local/lib/python3.7/dist-packages/pipreqs/pipreqs.py", line 409, in init
    follow_links=follow_links)
  File "/usr/local/lib/python3.7/dist-packages/pipreqs/pipreqs.py", line 138, in get_all_imports
    raise exc
  File "/usr/local/lib/python3.7/dist-packages/pipreqs/pipreqs.py", line 124, in get_all_imports
    tree = ast.parse(contents)
  File "/usr/lib/python3.7/ast.py", line 35, in parse
    return compile(source, filename, mode, PyCF_ONLY_AST)
  File "<unknown>", line 1
    !pip install uvicorn fastapi
    ^
SyntaxError: invalid syntax


In [2]:
!nvidia-smi

Sun Apr 11 06:20:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
gpt2.download_gpt2(model_name="124M")


Fetching checkpoint: 1.05Mit [00:00, 338Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 4.33Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 498Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:07, 70.2Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 259Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 5.87Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 9.94Mit/s]


In [4]:
gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load Data set

In [5]:
file_name = "role_dataset.txt"

In [6]:
netflix_data = pd.read_excel('/content/drive/My Drive/JobDescriptionPrediction/Resources/Data/processed/roles_dataset.xlsx')
netflix_data[['JobID','Job_Title','Output']].head(n=2)

,JobID,Job_Title,Output
0,9,bioinformatics data scientist,"['when applying, we ask that you reference any..."
1,13,data scientist,"['so if that gets you excited, apply!, role ex..."


In [7]:
TRAIN_SIZE      = 0.8
def split_data(netflix_data, S=TRAIN_SIZE):
    print(TRAIN_SIZE)

    # Split into training and validation sets    
    train_size = int(S * len(netflix_data))


    train_data = netflix_data[:train_size]
    val_data = netflix_data[train_size:]

    return train_data, val_data

In [8]:
train_data, val_data = split_data(netflix_data, TRAIN_SIZE)

f'There are {len(train_data) :,} samples for training, and {len(val_data) :,} samples for validation testing'

0.8


'There are 1,417 samples for training, and 355 samples for validation testing'

In [9]:
train_data.to_csv('role_dataset.txt', header=True, index=False, sep='\t', mode='w')

In [10]:

# gpt2.copy_file_from_gdrive("/content/drive/My Drive/JobDescriptionPrediction/Resources/Data/processed/role_dataset.txt")

## Finetune GPT-2


In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='run2',
              print_every=10,
              sample_every=200,
              save_every=100
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:03<00:00,  3.79s/it]


dataset has 787996 tokens
Training...
[10 | 18.89] loss=3.32 avg=3.32
[20 | 31.33] loss=3.28 avg=3.30
[30 | 43.77] loss=3.26 avg=3.28
[40 | 56.19] loss=2.96 avg=3.20
[50 | 68.63] loss=2.89 avg=3.14
[60 | 81.06] loss=3.15 avg=3.14
[70 | 93.49] loss=2.99 avg=3.12
[80 | 105.93] loss=2.83 avg=3.08
[90 | 118.35] loss=2.70 avg=3.04
[100 | 130.78] loss=2.69 avg=3.00
Saving checkpoint/run2/model-100
[110 | 145.88] loss=2.48 avg=2.95
[120 | 158.32] loss=3.06 avg=2.96
[130 | 170.77] loss=2.71 avg=2.94
[140 | 183.19] loss=2.71 avg=2.92
[150 | 195.64] loss=2.71 avg=2.91
[160 | 208.08] loss=2.75 avg=2.90
[170 | 220.51] loss=3.02 avg=2.91
[180 | 232.94] loss=2.77 avg=2.90
[190 | 245.37] loss=2.48 avg=2.87
[200 | 257.81] loss=2.15 avg=2.83
Saving checkpoint/run2/model-200
Instructions for updating:
Use standard file APIs to delete files with this prefix.
======== SAMPLE 1 ========
istical, predictive, sentiment analysis and data modeling a plus but there is a requirement to have some hands on experie

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='run2')

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint


In [ ]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-1000


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [ ]:
gpt2.generate(sess, run_name='run1')

advertising

"they're just getting started!" - one of the founders of aws, you might have heard, has just turned 50 (not quite yet eligible for a government sponsorship program yet).',#"bashononon" says the title of this post, but google will clearly identify the title on its support forums and in the google docs once this role has been over).',#"#bashononon" is a good day to be a software engineer in aws, aws cloud, or any other big data project, especially one that involves data at scale     some background in hadoop and some familiarity with hive       aws is a fantastic company to work with      #bashononon" says the title of this post, but google will clearly identify the title on its support forums and in the google docs once this role has been over).]']"
5126	seniormap data engineer	hive machine learning hadoop r sql data warehouse s3 spark kubernetes git nosql python aws	"['this role reports to the vp of hardware engineering and builds the infrastructure for continuous integrat

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [ ]:
gpt2.generate(sess,
              prefix="data science sap sql",
              length=150,
              temperature=0.7,
              nsamples=5
              )

data science sap sql json phd or higher in a quantitative or analytical field   3 years of experience with statistical or machine learning modeling skills (e.g., regression, support vector machine, clustering, etc.)', '4 years of experience with using tableau, or experience with a similar technology (e.g., r, sas, or matlab,    experience with object-oriented programming, query optimization, data integration (sql, hive, sql server),    experience with bi tools (e.g.', 'tableau),    experience with statistical packages (e.g., r, sas, or matlab,    experience with object-oriented programming, query optimization, data integration (sql,
data science sap sql tableau	"['you will be a part of a new initiative called machine learning at elecive, and will be working on machine learning with a wide variety of data sources from various sources, including social networks, email, search engine results, news, and internal external data.', 'as the principal data scientist, you will report to the chie

In [ ]:
gpt2.generate(sess,
              prefix="Associate data scientist AI Machine Learning Network R SAS C/C++ Java SPSS Data SciencePython",
              length=150,
              temperature=0.7,
              nsamples=5
              )

Associate data scientist AI Machine Learning Network R SAS C/C++ Java SPSS Data SciencePython sql .net java matlab data warehouse oop docker ci software development mvc	"[""lead the development of predictive analytics solutions, including machine learning, regression, and statistical methods by creating and maintaining data products and reporting."", 'you will provide insights from your data and your reporting into the business where the reporting is coming from., you will lead the use of statistical analysis, machine learning and predictive modeling to support our decision support initiatives across the economy.', ""you will be responsible for identifying and analyzing trends, identifying areas for future projects, and providing insights from these trends., you will be responsible for delivering insights from our data so that we can make more informed investment decisions., you will partner with clients to identify areas of opportunity, recommend
Associate data scientist AI Machine Le

In [ ]:
output = gpt2.generate(sess,
              prefix="data science AI Quantitative Analysis Data Mining Machine Learning Analysis Skills CSS",
              length=150,
              include_prefix=False,
              truncate=False,
              temperature=0.7,
              nsamples=2
              # ,
              # batch_size=5
              )

data science AI Quantitative Analysis Data Mining Machine Learning Analysis Skills CSS (manual search)
3.1 leadership skills, experience in leadership, strategic thinking, and problem-solving.', 'experience in managing and leading project teams and managing projects across three or more analytical disciplines, including research and development of analytical approaches and data models.', 'experience in managing and leading project teams and managing projects across three or more analytical disciplines, including research and development of analytical approaches and data models.', 'experience in managing and leading project teams and managing projects across three or more analytical disciplines, including research and development of analytical approaches and data models.', 'experience in managing and leading project teams and managing projects across three or more analytical disciplines, including research and development of analytical approaches and data models.',
data science AI Quant

In [ ]:
for i, output in enumerate(output):
    print(i)
    print(output)
    # text = tokenizer.decode(output, skip_special_tokens=True)
    # a = len("data science sap sql java")
    # print("{}: {}\n\n".format(i+1,  text[a:]))

TypeError: ignored

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [ ]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

In [ ]:
# may have to run twice to get file to download
files.download(gen_file)

## Generate Text From The Pretrained Model

If you want to generate text from the pretrained model, not a finetuned model, pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`.

This is currently the only way to generate text from the 774M or 1558M models with this notebook.

In [ ]:
# model_name = "774M"
model_name = "run1"

gpt2.download_gpt2(model_name=model_name)

Fetching checkpoint: 1.05Mit [00:00, 334Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 290Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 928Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.05Mit [00:00, 454Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 404Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 413Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 392Mit/s]                                                       


In [ ]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name=model_name)

In [ ]:
output = gpt2.generate(sess,
              model_name=model_name,
              prefix="data science sap sql java",
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=5
              # ,
              # batch_size=5
              )

JSONDecodeError: ignored

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.